# Загружаем библиотеки

In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 8.2MB 5.6MB/s 


In [5]:
from pymorphy2 import MorphAnalyzer

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import * 

# Читаем данные

[Датасет с твитами на русском языке](http://study.mokoron.com/) с разметкой *positive/negative*.

Читаем датафрейм с позитивными твитами, затем с негативными

In [6]:
tweets_positive = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/комп линг/tweets sentiment dataset/positive.csv',
                              sep=';', header=None, 
                              names=['id', 'date', 'author', 'text', 'type', 'replies', 'added_to_faves', 'author_tweets_count', 'retweets', 'followers','followed', 'lists'])

In [7]:
print(tweets_positive.shape)
tweets_positive['type'].unique()

(114911, 12)


array([1])

In [8]:
tweets_negative = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/комп линг/tweets sentiment dataset/negative.csv',
                              sep=';', header=None, 
                              names=['id', 'date', 'author', 'text', 'type', 'replies', 'added_to_faves', 'author_tweets_count', 'retweets', 'followers','followed', 'lists'])

In [9]:
print(tweets_negative.shape)
tweets_negative['type'].unique()

(111923, 12)


array([-1])

Датасеты очень большие (около 100 000 строк каждый), поэтому возьмем часть данных - по 25 000 твитов с позитивной и негативной меткой

In [10]:
tweets = tweets_positive.head(25000).append(tweets_negative.head(25000))

In [11]:
print(tweets.shape)
tweets['type'].unique()

(50000, 12)


array([ 1, -1])

Оставляем только столбцы *text* (текст твита) и *type* (индикатор разметки) - с ними будем работать.

In [19]:
df = tweets[['text', 'type']]
df.head()

,text,type
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


# Предобработка

Очищаем от тегов других пользователей и лишних пробельных символов.

In [20]:
df['text'].replace('(RT )?@[\w]*:?','', regex=True, inplace = True)
df['text'].replace('\s',' ', regex=True, inplace = True)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,text,type
0,"хоть я и школота, но поверь, у нас то же само...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,Ну ты идиотка) я испугалась за тебя!!!,1
3,"""Кто то в углу сидит и погибает от голода, а ...",1
4,"Вот что значит страшилка :D Но блин,посмотрев...",1


In [22]:
corpus = df['text'].to_list()
corpus[:5]

[' хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
 'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D',
 ' Ну ты идиотка) я испугалась за тебя!!!',
 ' "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…',
 ' Вот что значит страшилка :D Но блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D']

Токенизация, приведение к нижнему регистру и удаление знаков препинания

In [15]:
punct_marks = string.punctuation + "—" + "«" + "»" + "`" + "``"

In [23]:
for i in range(len(corpus)):
  doc = corpus[i]
  doc = word_tokenize(doc)
  doc = [word.lower() for word in doc if word not in punct_marks]
  corpus[i] = doc

corpus[0]

['хоть',
 'я',
 'и',
 'школота',
 'но',
 'поверь',
 'у',
 'нас',
 'то',
 'же',
 'самое',
 'd',
 'общество',
 'профилирующий',
 'предмет',
 'типа']

Лемматизация и удаление стоп-слов: 
* предлогов
* союзов
* частиц
* междометий 
* местоимений

In [24]:
morph = MorphAnalyzer()

In [25]:
corpus_lemmatized = []

for i in range(len(corpus)):
  doc = corpus[i]
  doc_lemmatized = []
  for word in doc:
    p = morph.parse(word)[0]
    if p.tag.POS is None or \
      p.tag.POS in ('PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO') or \
      'Anph' in p.tag or 'Apro' in p.tag:
      continue
    doc_lemmatized.append(p.normal_form)
  corpus_lemmatized.append(doc_lemmatized)

In [26]:
corpus_lemmatized[:5]

[['школотый', 'общество', 'профилировать', 'предмет', 'тип'],
 ['немного', 'похожий', 'мальчик', 'хороший'],
 ['идиотка', 'испугаться'],
 ['угол',
  'сидеть',
  'погибать',
  'голод',
  'ещё',
  'порция',
  'взять',
  'уже',
  'жрать',
  'хотеть'],
 ['страшилка',
  'посмотреть',
  'часть',
  'создаться',
  'ощущение',
  'автор',
  'курить',
  'что-то']]

Сохраним лемматизированные документы в виде строк, чтобы их мог принять векторизатор

In [27]:
X = [' '.join(text) for text in corpus_lemmatized]
X[0]

'школотый общество профилировать предмет тип'

Разделим выборку на обучающую и тестовую

In [28]:
y = df['type']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Мешок слов

Векторизуем тренировочные и тестовые данные

In [32]:
vectorizer = CountVectorizer(ngram_range=(1, 3))

In [59]:
X_train_vectorized = vectorizer.fit_transform(X_train)
list(vectorizer.vocabulary_.items())[:10]

[('быть', 18380),
 ('жуйковке', 76424),
 ('хабаровск', 285553),
 ('май', 117123),
 ('говориться', 45169),
 ('семейный', 228720),
 ('обстоятельство', 154322),
 ('жаль', 72716),
 ('причина', 202929),
 ('уважитель', 270894)]

Обучим модель и выведем матрицу с оценками качества классификации

In [60]:
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [61]:
X_test_vectorized = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.70      0.73      0.71      8244
           1       0.72      0.69      0.71      8256

    accuracy                           0.71     16500
   macro avg       0.71      0.71      0.71     16500
weighted avg       0.71      0.71      0.71     16500



Обучим несколько моделей с разным диапазоном n и сохраним значения метрик качества в датафрейм

In [68]:
eval_data = {'vectorizer': [], 'parameters': [], 
             'precision': [], 'recall': [],
             'accuracy': [], 'f_measure': []}

In [69]:
n_ranges = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]

for n_range in n_ranges:
  vectorizer = CountVectorizer(ngram_range=n_range)
  X_train_vectorized = vectorizer.fit_transform(X_train)

  clf.fit(X_train_vectorized, y_train)

  X_test_vectorized = vectorizer.transform(X_test)
  y_pred = clf.predict(X_test_vectorized)

  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  accuracy = accuracy_score(y_test, y_pred)
  f_measure = f1_score(y_test, y_pred)

  eval_data['vectorizer'].append('ngrams')
  eval_data['parameters'].append(n_range)
  eval_data['precision'].append(precision)
  eval_data['recall'].append(recall)
  eval_data['accuracy'].append(accuracy)
  eval_data['f_measure'].append(f_measure)

# Мешок символьных n-грамм

Аналогичным образом обучим модель с использованием символьных n-грамм

In [70]:
n_ranges = [(2, 8), (3, 8), (4,8), (5, 8)]

for n_range in n_ranges:
  char_vectorizer = CountVectorizer(analyzer='char', ngram_range=n_range)
  X_train_vectorized = char_vectorizer.fit_transform(X_train)

  clf = MultinomialNB()
  clf.fit(X_train_vectorized, y_train)

  X_test_vectorized = char_vectorizer.transform(X_test)
  y_pred = clf.predict(X_test_vectorized)

  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  accuracy = accuracy_score(y_test, y_pred)
  f_measure = f1_score(y_test, y_pred)

  eval_data['vectorizer'].append('nchars')
  eval_data['parameters'].append(n_range)
  eval_data['precision'].append(precision)
  eval_data['recall'].append(recall)
  eval_data['accuracy'].append(accuracy)
  eval_data['f_measure'].append(f_measure)

# Tf-Idf векторизация

Повторим то же самое с использованием Tf-Idf векторизации

In [71]:
n_ranges = [(1, 3), (1, 4)]
max_df_list = [0.95]
min_df_list = [1, 2, 3, 4]

for n_range in n_ranges:
  for max_df in max_df_list:
    for min_df in min_df_list:
      tfidf_vectorizer = TfidfVectorizer(ngram_range=n_range, max_df = max_df, min_df = min_df)
      X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)

      clf = MultinomialNB()
      clf.fit(X_train_vectorized, y_train)

      X_test_vectorized = tfidf_vectorizer.transform(X_test)
      y_pred = clf.predict(X_test_vectorized)

      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      accuracy = accuracy_score(y_test, y_pred)
      f_measure = f1_score(y_test, y_pred)

      eval_data['vectorizer'].append('tf-idf')
      eval_data['parameters'].append((n_range, max_df, min_df))
      eval_data['precision'].append(precision)
      eval_data['recall'].append(recall)
      eval_data['accuracy'].append(accuracy)
      eval_data['f_measure'].append(f_measure)

# Сравнение метрик качества

Сохраним значения метрик качества в виде датафрейма

In [72]:
eval_data = pd.DataFrame(eval_data)
eval_data

,vectorizer,parameters,precision,recall,accuracy,f_measure
0,ngrams,"(1, 1)",0.705912,0.688313,0.701758,0.697001
1,ngrams,"(1, 2)",0.707429,0.693664,0.704364,0.700479
2,ngrams,"(1, 3)",0.706559,0.694272,0.703939,0.700362
3,ngrams,"(1, 4)",0.706574,0.694029,0.703879,0.700245
4,ngrams,"(1, 5)",0.706743,0.693421,0.703818,0.700018
5,ngrams,"(1, 6)",0.707175,0.692813,0.703939,0.699920
6,nchars,"(2, 8)",0.708096,0.679679,0.700727,0.693596
7,nchars,"(3, 8)",0.708730,0.678220,0.700727,0.693139
8,nchars,"(4, 8)",0.708360,0.679071,0.700727,0.693406
9,nchars,"(5, 8)",0.705882,0.674206,0.697636,0.689681


Выведем три лучших векторизатора по F-мере

In [73]:
eval_data.sort_values(by='f_measure', ascending=False).iloc[:3]

,vectorizer,parameters,precision,recall,accuracy,f_measure
15,tf-idf,"((1, 4), 0.95, 2)",0.680074,0.758726,0.701879,0.717250
11,tf-idf,"((1, 3), 0.95, 2)",0.684030,0.753740,0.703758,0.717195
16,tf-idf,"((1, 4), 0.95, 3)",0.681557,0.734768,0.696727,0.707163


Три лучших векторизатора по precision

In [74]:
eval_data.sort_values(by='precision', ascending=False).iloc[:3]

,vectorizer,parameters,precision,recall,accuracy,f_measure
14,tf-idf,"((1, 4), 0.95, 1)",0.713439,0.688192,0.706848,0.700588
10,tf-idf,"((1, 3), 0.95, 1)",0.712635,0.690016,0.706848,0.701143
7,nchars,"(3, 8)",0.708730,0.678220,0.700727,0.693139


Три лучших векторизатора по recall

In [75]:
eval_data.sort_values(by='recall', ascending=False).iloc[:3]

,vectorizer,parameters,precision,recall,accuracy,f_measure
15,tf-idf,"((1, 4), 0.95, 2)",0.680074,0.758726,0.701879,0.717250
11,tf-idf,"((1, 3), 0.95, 2)",0.684030,0.753740,0.703758,0.717195
16,tf-idf,"((1, 4), 0.95, 3)",0.681557,0.734768,0.696727,0.707163


Выведем лидеров по набору параметров для каждого вида векторизатора

In [77]:
eval_data.groupby('vectorizer').apply(
    lambda x: x.sort_values(by='f_measure', ascending=False).head(1))

,,vectorizer,parameters,precision,recall,accuracy,f_measure
vectorizer,,,,,,,
nchars,6,nchars,"(2, 8)",0.708096,0.679679,0.700727,0.693596
ngrams,1,ngrams,"(1, 2)",0.707429,0.693664,0.704364,0.700479
tf-idf,15,tf-idf,"((1, 4), 0.95, 2)",0.680074,0.758726,0.701879,0.717250


Как можно видеть, наиболее высокий результат дает **векторизатор tf-idf** с диапазоном размера **n-грамм (1,4)**: 
* **F-мера = 0.717** 
* **recall = 0.725**
* **precision** не очень сильно отклоняется от максимально возможного значения 0.71 (здесь 0.68)